In [69]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import numpy as np
from scipy import stats
import seaborn as sns
from functools import reduce
pd.options.mode.chained_assignment = None 
from statistics import mean
from Profile_Generation import *
from plotting import *
import warnings
warnings.simplefilter('ignore', np.RankWarning)
from scipy.optimize import curve_fit
import matplotlib.patches as patches
from scipy.optimize import curve_fit
from openpyxl import load_workbook
import math
from scipy.optimize import OptimizeWarning
warnings.simplefilter("ignore", OptimizeWarning)
import timeit
from sklearn.metrics import mean_squared_error
from matplotlib.pyplot import cm
from matplotlib.patches import Patch
from scipy import stats
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.stats import levene
import sys

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
%%time
# ~5mins

# Execute CURATE without pop tau
execute_CURATE(pop_tau_string='')

finished patient 84
finished patient 114
Patient #117 has insufficient/<3 predictions (1 predictions) (for linear)!
Patient #117 has insufficient/<3 predictions (0 predictions) (for quadratic)!
finished patient 117
finished patient 118
finished patient 120
Patient #121 has insufficient unique dose-response pairs for calibration (for quad)!
finished patient 121
finished patient 122
finished patient 123
finished patient 125
Patient #126 has insufficient unique dose-response pairs for calibration (for quad)!
finished patient 126
finished patient 129
Patient #130 has insufficient/<3 predictions (2 predictions) (for linear)!
Patient #130 has insufficient/<3 predictions (1 predictions) (for quadratic)!
finished patient 130
finished patient 131
finished patient 132
Patient #133 has insufficient/<3 predictions (0 predictions) (for linear)!
Patient #133 has insufficient unique dose-response pairs for calibration (for quad)!
finished patient 133
finished patient 138
Patients to exclude for linea

In [24]:
all_data()

,day,response,dose_mg,patient,ideal,body_weight,dose
0,2.0,2.4,0.5,84,True,9.70,0.051546
1,3.0,2.8,1.0,84,True,9.70,0.103093
2,4.0,3.2,1.5,84,True,9.70,0.154639
3,5.0,3.1,1.5,84,True,9.70,0.154639
4,6.0,7.9,3.0,84,True,9.70,0.309278
...,...,...,...,...,...,...,...
447,26.0,9.3,NaN,138,False,18.45,NaN
448,27.0,NaN,NaN,138,False,18.45,NaN
449,28.0,NaN,NaN,138,False,18.45,NaN
450,29.0,NaN,NaN,138,False,18.45,NaN


,patient,acceptable
count,16.000000,16.000000
mean,122.687500,43.483967
std,12.229575,18.746935
min,84.000000,13.793103
25%,119.500000,29.107143
50%,124.000000,49.076355
75%,130.250000,57.512315
max,138.000000,71.428571


In [70]:
def clinically_relevant_performance_metrics():
    """Clinically relevant performance metrics. 
    Calculate the results, conduct statistical tests, and
    print them out. 
    
    Instructions: Uncomment first block of code to write output to txt file.
    """
    # Uncomment to write output to txt file
    # file_path = 'Clinically relevant performance metrics.txt'
    # sys.stdout = open(file_path, "w")

    # Clinically relevant performance metrics. 
    # Calculate the results, conduct statistical tests, and
    # print them out. 

    # 1. Find percentage of days within clinically acceptable 
    # tac levels (6.5 to 12 ng/ml)

    data = pd.read_excel(all_data_file)

    # Add acceptable tacrolimus levels column
    data['acceptable'] = (data['response'] >= acceptable_tac_lower_limit) & (data['response'] <= acceptable_tac_upper_limit)

    # Calculate results
    acceptable_SOC = \
    data.groupby('patient')['acceptable'].apply(lambda x: x.sum()/x.count()*100).to_frame().reset_index()
    shapiro_p = stats.shapiro(acceptable_SOC.acceptable).pvalue
    if shapiro_p < 0.05:
        shapiro_result_string = 'reject normality'
    else:
        shapiro_result_string = 'assume normality'

    # Print results and normality test results
    print(f'(1) % of days within clinically acceptable tac levels:\n{acceptable_SOC.acceptable.describe()}\n')
    print(f'p-value = {shapiro_p:.2f}, Shapiro-Wilk test, {shapiro_result_string}')

    # 2. Find % of predictions within clinically acceptable
    # prediction error (between -1.5 and +2 ng/ml)

    dat = pd.read_excel(result_file, sheet_name='result')
    dat = dat[dat.method=='L_RW_wo_origin'].reset_index()
    dat = dat[['patient', 'pred_day', 'deviation']]

    # Add acceptable therapeutic range column
    dat['acceptable'] = (dat['deviation'] >=overprediction_limit) & (dat['deviation'] <= underprediction_limit)

    # Calculate results
    acceptable_CURATE = \
    dat.groupby('patient')['acceptable'].apply(lambda x: x.sum()/x.count()*100).to_frame().reset_index()
    shapiro_p = stats.shapiro(acceptable_CURATE.acceptable).pvalue
    if shapiro_p < 0.05:
        shapiro_result_string = 'reject normality'
    else:
        shapiro_result_string = 'assume normality'

    print(f'\n(2) % of predictions within clinically acceptable prediction error:\n{acceptable_CURATE.acceptable.describe()}\n')
    print(f'p-value = {shapiro_p:.2f}, Shapiro-Wilk test, {shapiro_result_string}\n')

    # Check for equal variance between (1) and (2): result p = 0.97, assume equal variance
    stat, p = levene(acceptable_SOC.acceptable, acceptable_CURATE.acceptable, center='mean')
    if p < 0.05:
        levene_result_string = 'unequal variance'
    else:
        levene_result_string = 'assume equal variance'
    print(f'p-value = {p:.2f}, Levene test, {levene_result_string}\n')

    # Check for equal means
    t_test = stats.ttest_ind(acceptable_SOC.acceptable, acceptable_CURATE.acceptable).pvalue
    if p < 0.05:
        result_string = 'unequal means'
    else:
        result_string = 'assume equal means'

    print(f'Comparison of means between (1) and (2), p-value = {p:.2f}, {result_string}\n')

    # 3. Clinically unacceptable overprediction

    # Add unacceptable overprediction
    dat['unacceptable_overprediction'] = (dat['deviation'] < overprediction_limit)
    dat['unacceptable_underprediction'] = (dat['deviation'] > underprediction_limit)

    unacceptable_overprediction = \
    dat.groupby('patient')['unacceptable_overprediction'].apply(lambda x: x.sum()/x.count()*100).to_frame().reset_index()
    result = stats.shapiro(unacceptable_overprediction.unacceptable_overprediction).pvalue
    if result < 0.05:
        result_string = 'reject normality'
    else:
        result_string = 'assume normality'

    print(f'(3) % clinically unacceptable overprediction:\n{unacceptable_overprediction.unacceptable_overprediction.describe()}\n')
    print(f'p-value = {result:.2f}, Shapiro-Wilk test, {result_string}\n')

    # 4. Clinically unacceptable underprediction

    # Add unacceptable underprediction
    unacceptable_underprediction = \
    dat.groupby('patient')['unacceptable_underprediction'].apply(lambda x: x.sum()/x.count()*100).to_frame().reset_index()
    result = stats.shapiro(unacceptable_underprediction.unacceptable_underprediction).pvalue
    if result < 0.05:
        result_string = 'reject normality'
    else:
        result_string = 'assume normality'

    print(f'(4) % clinically unacceptable underprediction:\n{unacceptable_underprediction.unacceptable_underprediction.describe()}\n')
    print(f'p-value = {result:.2f}, Shapiro-Wilk test, {result_string}\n')

0.23814751207828522
count    13.000000
mean     24.433002
std      17.144551
min       0.000000
25%      17.647059
50%      25.000000
75%      37.500000
max      50.000000
Name: unacceptable_overprediction, dtype: float64
0.6183906197547913
count    13.000000
mean     22.956031
std      13.587987
min       0.000000
25%      18.181818
50%      23.529412
75%      28.571429
max      44.444444
Name: unacceptable_underprediction, dtype: float64


In [60]:
dat = pd.read_excel('CURATE_results.xlsx', sheet_name='result')

In [62]:
df = dat.copy()
df.groupby('method')['deviation'].describe()

,count,mean,std,min,25%,50%,75%,max
method,,,,,,,,
L_Cum_origin_dp,117.0,-0.358863,2.850133,-9.262085,-2.437500,-0.469231,1.266667,9.729032
L_Cum_origin_dp_tau,4446.0,-0.023743,2.738046,-9.419765,-1.787283,-0.380046,1.712226,15.697863
L_Cum_origin_int,117.0,-0.357066,2.813285,-8.055000,-2.384677,-0.511111,1.303478,9.460000
L_Cum_origin_int_tau,4446.0,0.081629,2.981882,-9.942010,-1.804167,-0.299762,1.828948,15.697865
L_Cum_wo_origin,117.0,-0.357066,2.813285,-8.055000,-2.384677,-0.511111,1.303478,9.460000
L_Cum_wo_origin_tau,4446.0,0.081629,2.981882,-9.942010,-1.804167,-0.299762,1.828948,15.697865
L_PPM_origin_dp,117.0,-0.224887,2.405828,-7.800000,-1.714286,-0.464286,1.142857,5.900000
L_PPM_origin_dp_tau,4446.0,-0.192291,2.388893,-7.526207,-1.758194,-0.584454,1.227048,8.092282
L_PPM_origin_int,117.0,-0.038889,3.127148,-8.800000,-1.900000,-0.600000,2.000000,8.900000


In [57]:
52.61-43.8

8.810000000000002

,patient,acceptable
count,13.000000,13.000000
mean,121.769231,52.610967
std,13.039388,21.719827
min,84.000000,21.428571
25%,120.000000,35.714286
50%,123.000000,54.545455
75%,129.000000,60.000000
max,138.000000,100.000000


In [20]:
input_file = 'Retrospective Liver Transplant Data - edited.xlsx'
rows_to_skip = 17
list_of_patient_df = []
patient = '84'

df = pd.read_excel(input_file, sheet_name=patient, skiprows=rows_to_skip)
df = clean_data(df)
# df = keep_ideal_data(df, patient, list_of_patient_df)
original_df = df.copy()
df_temp = df.copy()

# Create boolean column of data to remove
# Including NA, <2 tac level, multiple blood draws
df_temp['non_ideal'] = (df_temp.isnull().values.any(axis=1))  | \
                           (df_temp['Tac level (prior to am dose)'] == '<2') | \
                           (df_temp["Tac level (prior to am dose)"].astype(str).str.contains("/"))

# df_temp
# Set boolean for non_ideal as True if all dose including and above current row is 0
for i in range(len(df_temp)):
    if (df_temp.loc[0:i, 'Eff 24h Tac Dose'] == 0).all():
        df_temp.loc[i, 'non_ideal'] = True

# Create index column
df_temp.reset_index(inplace=True) 

# Find cumulative sum of data to be removed for each index row
df_cum_sum_non_ideal = df_temp['non_ideal'].cumsum()     

# # Find number of consecutive non-NA
# df_temp = df_temp.groupby(df_cum_sum_non_ideal).agg({'index': ['count', 'min', 'max']})

# # Groupby created useless column level for index, drop it
# df_temp.columns = df_temp.columns.droplevel()

# # Find largest chunk with consec non-NA
# df_temp = df_temp[df_temp['count']==df_temp['count'].max()] 
# df_temp.reset_index(inplace=True)

# # Find index of largest chunk to keep in dataframe
# if len(df_temp) > 1: # If there are >1 large chunks with longest length, an error will be printed
#     df_temp = print("there are >1 chunks of data with the longest length.")
# else:
#     # Find index of largest chunk to keep in dataframe
#     min_idx = df_temp.loc[0, 'min']
#     max_idx = df_temp.loc[0, 'max'] # Get max index where non-NA chunk ends

#     # Keep largest chunk in dataframe
#     df = df.iloc[min_idx:max_idx + 1, :] 

# # Format patient dataframe
# df['patient'] = patient
# df.columns = ['day', 'response', 'dose', 'patient']

print(f'df_cum_sum_non_ideal\n {df_cum_sum_non_ideal} |\ndf_temp {df_temp}')

df_cum_sum_non_ideal
 0      0
1      0
2      0
3      0
4      0
5      0
6      0
7      1
8      2
9      3
10     4
11     4
12     4
13     4
14     5
15     6
16     6
17     7
18     8
19     9
20    10
21    11
22    12
23    13
24    14
25    15
26    16
27    17
Name: non_ideal, dtype: int32 |
df_temp     index  Day # Tac level (prior to am dose)  Eff 24h Tac Dose  non_ideal
0       0      2                          2.4               0.5      False
1       1      3                          2.8               1.0      False
2       2      4                          3.2               1.5      False
3       3      5                          3.1               1.5      False
4       4      6                          7.9               3.0      False
5       5      7                           10               3.0      False
6       6      8                         10.3               3.0      False
7       7      9                          NaN               3.0       True
8       8  

In [ ]:
all_data()

In [ ]:
df = pd.read_excel('Retrospective Liver Transplant Data - edited.xlsx', sheet_name='84', skiprows=17)

# Keep target columns
df = df[["Day #", "Tac level (prior to am dose)", "Eff 24h Tac Dose"]]

# Shift tac level one cell up to match dose-response to one day
df['Eff 24h Tac Dose'] = df['Eff 24h Tac Dose'].shift(1)

# Remove "mg"/"ng" from dose
df['Eff 24h Tac Dose'] = df['Eff 24h Tac Dose'].astype(str).str.replace('mg', '')
df['Eff 24h Tac Dose'] = df['Eff 24h Tac Dose'].astype(str).str.replace('ng', '')
df['Eff 24h Tac Dose'] = df['Eff 24h Tac Dose'].astype(float)

first_day_of_dosing = df['Day #'].loc[~df['Eff 24h Tac Dose'].isnull()].iloc[0]

# Keep data from first day of dosing
df = df[df['Day #'] >= first_day_of_dosing].reset_index(drop=True)

# Set the first day of dosing as day 2 (because first dose received on day 1)
for i in range(len(df)):
    df.loc[i, 'Day #'] = i + 2

original_df = df.copy()
df_temp = df.copy()

# Create boolean column of data to remove
# Including NA, <2 tac level, multiple blood draws
df_temp['non_ideal'] = (df_temp.isnull().values.any(axis=1))  | \
                           (df_temp['Tac level (prior to am dose)'] == '<2') | \
                           (df_temp["Tac level (prior to am dose)"].astype(str).str.contains("/"))

# Set boolean for non_ideal as True if all dose including and above current row is 0
for i in range(len(df_temp)):
    if (df_temp.loc[0:i, 'Eff 24h Tac Dose'] == 0).all():
        df_temp.loc[i, 'non_ideal'] = True

# Create index column
df_temp.reset_index(inplace=True) 

# Find cumulative sum of data to be removed for each index row
df_cum_sum_non_ideal = df_temp['non_ideal'].cumsum()     

# Find number of consecutive non-NA
df_temp = df_temp.groupby(df_cum_sum_non_ideal).agg({'index': ['count', 'min', 'max']})

# Groupby created useless column level for index, drop it
df_temp.columns = df_temp.columns.droplevel()

# Find largest chunk with consec non-NA
df_temp = df_temp[df_temp['count']==df_temp['count'].max()] 
df_temp.reset_index(inplace=True)

# Find index of largest chunk to keep in dataframe
if len(df_temp) > 1: # If there are >1 large chunks with longest length, an error will be printed
    df_temp = print("there are >1 chunks of data with the longest length.")
else:
    # Find index of largest chunk to keep in dataframe
    min_idx = df_temp.loc[0, 'min'] 
    max_idx = df_temp.loc[0, 'max'] # Get max index where non-NA chunk ends

    # Keep largest chunk in dataframe
    df = df.iloc[min_idx:max_idx + 1, :] 

# Format patient dataframe
df['patient'] = patient
df.columns = ['day', 'response', 'dose', 'patient']

print(f'df_cum_sum_non_ideal\n {df_cum_sum_non_ideal} \n df\n {df_temp} \n min_idx: {min_idx} | max_idx {max_idx} \n df\n {df} \n original_df \n {original_df}')

In [ ]:
patient='84'

patient_df = pd.read_excel('Retrospective Liver Transplant Data - edited.xlsx', sheet_name=patient, skiprows=17)
patient_df['patient'] = patient

# Subset dataframe
patient_df = patient_df[['Day #', 'Tac level (prior to am dose)', 'Eff 24h Tac Dose', 'patient']]

# Shift dose column
patient_df['Eff 24h Tac Dose'] = patient_df['Eff 24h Tac Dose'].shift(1)

# Remove "mg"/"ng" from dose
patient_df['Eff 24h Tac Dose'] = patient_df['Eff 24h Tac Dose'].astype(str).str.replace('mg', '')
patient_df['Eff 24h Tac Dose'] = patient_df['Eff 24h Tac Dose'].astype(str).str.replace('ng', '')
patient_df['Eff 24h Tac Dose'] = patient_df['Eff 24h Tac Dose'].astype(float)

first_day_of_dosing = patient_df['Day #'].loc[~patient_df['Eff 24h Tac Dose'].isnull()].iloc[0]

# Keep data from first day of dosing
patient_df = patient_df[patient_df['Day #'] >= first_day_of_dosing].reset_index(drop=True)

# Set the first day of dosing as day 2 (because first dose received on day 1)
for i in range(len(patient_df)):
    patient_df.loc[i, 'Day #'] = i + 2
    
patient_df

In [ ]:
# Find % of days within clinically acceptable tac levels

df = read_excel('all_data.xlsx')
df

In [ ]:
%%time
# Perform CV
five_fold_cross_val_results, five_fold_cross_val_results_summary = find_pop_tau_with_CV()
execute_CURATE_and_update_pop_tau_results('CV', five_fold_cross_val_results_summary, five_fold_cross_val_results)

# Perform LOOCV
five_fold_cross_val_results, five_fold_cross_val_results_summary = find_pop_tau_with_LOOCV()
execute_CURATE_and_update_pop_tau_results('LOOCV', five_fold_cross_val_results_summary, five_fold_cross_val_results)